In [ ]:
PATH_TO_STORE_EXPERIMENTS = "data\\rl_training"

## Define RL Pipeline structure

In [ ]:
#from automl.base_configurations.environment.cart_pole import dqn_sb3 as base_rl_configuration
from automl.base_configurations.environment.cart_pole import ppo_sb3 as base_rl_configuration


rl_pipeline_config = base_rl_configuration.config_dict()

In [ ]:
print(rl_pipeline_config)

In [ ]:
#experiment_name = "dqn_sb3_cartpole"
#experiment_name = "dqn_sb3_cartpole_ppo"

experiment_name = "ppo_cartpole"

### Change Environment

In [ ]:
environment = rl_pipeline_config["input"]["environment"]
environment_input = environment[1]

#environment_input["render_mode"] = "human"

## Hyperparameters

### Base Model

In [ ]:
LOAD_MODEL = True

In [ ]:
rl_pipeline_input = rl_pipeline_config["input"]

rl_trainer_tuple = rl_pipeline_input["rl_trainer"]
rl_trainer_input = rl_trainer_tuple[1]

agents_input = rl_pipeline_input["agents_input"]

policy_tuple = agents_input["policy"]
policy_input = policy_tuple[1]

agents_trainers_input = rl_trainer_input["agents_trainers_input"]

In [ ]:
learner_tuple = agents_trainers_input["learner"]
learner_input = learner_tuple[1]

optimizer_tuple = learner_input["optimizer"]
optimizer_input = optimizer_tuple[1]

In [ ]:
import os

if LOAD_MODEL:

    #base_model_path = 'data\\models\\sb3_CartPole_dqn\\sb3_CartPole_dqn_perturbed_0_10'
    #base_model_path = 'data\\models\\sb3_CartPole_dqn\\sb3_CartPole_dqn_perturbed_5_50'
    base_model_path = 'data\\models\\sb3_CartPole_ppo\\sb3_CartPole_ppo_gaussian_0_0.8_0.9'

    #base_model_path = 'data\\models\\FC_CartPole_ppo\\FC_CartPole_ppo'
    
    from automl.utils.json_utils.shape_json_utils import CustomSpaceJsonEncoderDecoder
    
    model_name = os.path.basename(base_model_path)
    
    experiment_name = f"{experiment_name}\\{model_name}"
    
    rl_pipeline_input = rl_pipeline_config["input"]
    
    policy_input["model"] = base_model_path


In [ ]:
learner_input["critic_model"] = 'data\\models\\sb3_CartPole_ppo_critic\\sb3_CartPole_ppo_critic'

In [ ]:
#rl_trainer_input["limit_total_steps"] = 90000

#rl_trainer_input.pop("limit_total_steps", None)

#rl_trainer_input["num_episodes"] = 4000


In [ ]:
#agents_trainers_input["learning_start_step_delay"] = 5000
#agents_trainers_input["learning_start_ep_delay"] = 150

In [ ]:
#agents_trainers_input["optimization_interval"] = 2048
#agents_trainers_input["times_to_learn"] = 1

In [ ]:


#optimizer_input["clip_grad_norm"] = 0.1
#optimizer_input["clip_grad_value"] = 0.1
#optimizer_tuple = learner_input["learning_rate"] = 0.0012

## Gen RL Pipeline

In [ ]:

from automl.rl.rl_pipeline import RLPipelineComponent
from automl.utils.json_utils.json_component_utils import gen_component_from

rl_pipeline : RLPipelineComponent = gen_component_from(rl_pipeline_config)

In [ ]:
rl_pipeline.pass_input({"base_directory" : PATH_TO_STORE_EXPERIMENTS,
                        "artifact_relative_directory" : experiment_name,
                        "create_new_directory" : True})

experiment_path = rl_pipeline.get_artifact_directory()

print(f"Experiment path: {experiment_path}")

## Do the training

In [ ]:
from automl.loggers.global_logger import activate_global_logger

activate_global_logger(rl_pipeline.get_artifact_directory())

In [ ]:
rl_pipeline.run()

### Save configuration

In [ ]:
#rl_pipeline.save_configuration(save_exposed_values=True)
from automl.basic_components.state_management import save_state


save_state(rl_pipeline, save_definition=True)

## See Results

In [ ]:
AGGREGATE_NUMBER = 10

In [ ]:

from automl.loggers.result_logger import RESULTS_FILENAME, ResultLogger

results_directory = f"{experiment_path}\\RLTrainerComponent"
    
results_logger = ResultLogger(input={
                                        "results_filename" : RESULTS_FILENAME,
                                        "base_directory" : results_directory,
                                        "artifact_relative_directory" : '',
                                        "create_new_directory" : False
                            })

In [ ]:
#results_logger.plot_graph(x_axis='episode', y_axis=[('total_reward', name)], to_show=False)
results_logger.plot_confidence_interval(x_axis='episode', y_column='episode_reward',show_std=False, to_show=False, y_values_label=experiment_name, aggregate_number=AGGREGATE_NUMBER)
results_logger.plot_linear_regression(x_axis='episode', y_axis='episode_reward', to_show=False, y_label=experiment_name + '_linear')